# Coordinate Query

This notebook demonstrates how to query the survey for datacubes based on a coordinate.

In [2]:
import os.path as op
import numpy as np
from astropy.table import Table

from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.io import fits

from astropy.wcs import WCS

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

In [3]:
pdr1_dir = '/home/jovyan/Hobby-Eberly-Public/HETDEX/internal/pdr1/'
if not op.exists(pdr1_dir):
    pdr1_dir = 'pdr1/'
    
ifu_data = Table.read(op.join( pdr1_dir, 'ifu-index.fits'))

In [4]:
# Create SkyCoord Object array for IFU centers
ifu_coords = SkyCoord( ra=ifu_data['ra_cen']*u.deg, dec=ifu_data['dec_cen']*u.deg)

## Single Coordinate Search

This is an example of a big Lyman Alpha Blob at z=2.53. Check out notebook 07-CollapsingCubes.ipynb to show you how to create a narrowband line flux map of this object


In [5]:
coord = SkyCoord(ra=228.78581*u.deg, dec=51.268036*u.deg)

In [6]:
# find list of possible datacubes with coverage
sel = coord.separation( ifu_coords) < 26*u.arcsec

In [7]:
ifu_data[sel]

shotid,ifuslot,ra_cen,dec_cen,flag,flag_badamp,flag_badfib,flag_meteor,flag_satellite,flag_shot,flag_throughput,field,objid,date,obsid,ra_shot,dec_shot,pa,n_ifu,fwhm_virus,fwhm_virus_err,response_4540,ambtemp,datevobs,dewpoint,exptime,humidity,mjd,nstars_fit_fwhm,obsind,pressure,structaz,time,trajcdec,trajcpa,trajcra
int64,bytes3,float32,float32,float32,float32,float32,float32,float32,float32,float32,bytes12,bytes18,int32,int32,float64,float64,float64,int32,float32,float32,float32,float32,bytes12,float32,float32,float32,float32,int32,int32,float32,float32,bytes7,float32,float32,float32
20180618017,024,228.79074,51.264973,1.0,1.0,1.0,1.0,1.0,1.0,1.0,dex-spring,HS00444d2_W,20180618,17,228.871039,51.349437,292.50071,40,1.4246124,0.16366434,0.1826,18.612223,20180618v017,9.869445,367.6003,55.798,58287.285,19,1,804.809,317.00266,0648106,51.3479,290.8775,228.87


In [8]:
# print the data path to the datacubes
for row in ifu_data[sel]:
    
    shotid = row['shotid']
    ifuslot= row['ifuslot']
    datacube_path = op.join( pdr1_dir, 'datacubes', str(shotid), 'dex_cube_{}_{}.fits'.format( shotid, ifuslot))
    print( datacube_path)

/home/jovyan/Hobby-Eberly-Public/HETDEX/internal/pdr1/datacubes/20180618017/dex_cube_20180618017_024.fits


## AGN Catalog Search

In [9]:
sdss_agn = Table.read('dr16q_hdr5.fits')
                          
agn_coords = SkyCoord(ra = sdss_agn['ra'], dec= sdss_agn['dec'], unit='deg')

We will want all tiles that overlap with this list of coordinates not just the closest match. 
Astropy SkyCoord Class function "search_around_sky" is great for this. Here we will search 26 arcsec within every sources in the catalog list

In [10]:
idxc, idxcatalog, d2d, d3d = agn_coords.search_around_sky(ifu_coords, 26*u.arcsec)

Here we list the datapath of the cubes that overlap with coordinate catalog. To see how to access each of these cubes and extract spectra see 08-ExtractingSpectra.ipynb to decompress all datacubes in a list see 09-BatchDecompression.ipynb

In [11]:
for row in ifu_data[ idxc][0:15]:    
    shotid = row['shotid']
    ifuslot= row['ifuslot']
    datacube_path = op.join( pdr1_dir, 'datacubes', str(shotid), 'dex_cube_{}_{}.fits'.format( shotid, ifuslot))
    print( datacube_path)

/home/jovyan/Hobby-Eberly-Public/HETDEX/internal/pdr1/datacubes/20170222012/dex_cube_20170222012_083.fits
/home/jovyan/Hobby-Eberly-Public/HETDEX/internal/pdr1/datacubes/20170222012/dex_cube_20170222012_083.fits
/home/jovyan/Hobby-Eberly-Public/HETDEX/internal/pdr1/datacubes/20170222012/dex_cube_20170222012_083.fits
/home/jovyan/Hobby-Eberly-Public/HETDEX/internal/pdr1/datacubes/20170301015/dex_cube_20170301015_083.fits
/home/jovyan/Hobby-Eberly-Public/HETDEX/internal/pdr1/datacubes/20170301015/dex_cube_20170301015_083.fits
/home/jovyan/Hobby-Eberly-Public/HETDEX/internal/pdr1/datacubes/20170306018/dex_cube_20170306018_083.fits
/home/jovyan/Hobby-Eberly-Public/HETDEX/internal/pdr1/datacubes/20170306018/dex_cube_20170306018_083.fits
/home/jovyan/Hobby-Eberly-Public/HETDEX/internal/pdr1/datacubes/20170322011/dex_cube_20170322011_073.fits
/home/jovyan/Hobby-Eberly-Public/HETDEX/internal/pdr1/datacubes/20170322011/dex_cube_20170322011_073.fits
/home/jovyan/Hobby-Eberly-Public/HETDEX/intern